
The existing pipeline does not work for games not yet played. These do not show up on the NBA stats pages, so are not retrieved by the existing pipeline, and therefore are not present in the hopsworks feature store.

There does not appear to be a way to retrieve a record for these yet-to-be-played games from the NBA stats page, so they kind of have to be created from scratch.

Before the model can be applied to predict these games, a separate workflow is needed to get these unplayed games into the feature store:
 - find the scheduled games for the day 
 - construct a blank game record with specified home team id and visitor team id and add to a dataframe
 - pull historical data from previous games into the dataframe
 - generate features using previous game data (rolling averages, win streaks, etc…), 
 - add these new records to the feature store temporarily


In [1]:
import os

import pandas as pd
import numpy as np

from src.webscraping import (
    activate_web_driver,
    get_todays_matchups,
)

**Retrieve todays games**

In [2]:
#retrieve list of teams playing today

driver = activate_web_driver('firefox')

# get today's games on NBA schedule
teams_list = get_todays_matchups(driver)

team_count = len(teams_list) 
matchups = []
for i in range(0,team_count,2):
    matchups.append([teams_list[i],teams_list[i+1]])

matchups


[WDM] - Downloading: 19.0kB [00:00, 19.6MB/s]                   
[WDM] - Downloading: 100%|██████████| 1.58M/1.58M [00:00<00:00, 5.06MB/s]


[['/team/1610612743/nuggets/', '/team/1610612740/pelicans/'],
 ['/team/1610612756/suns/', '/team/1610612759/spurs/'],
 ['/team/1610612738/celtics/', '/team/1610612751/nets/'],
 ['/team/1610612763/grizzlies/', '/team/1610612765/pistons/'],
 ['/team/1610612739/cavaliers/', '/team/1610612752/knicks/'],
 ['/team/1610612747/lakers/', '/team/1610612764/wizards/'],
 ['/team/1610612741/bulls/', '/team/1610612758/kings/'],
 ['/team/1610612754/pacers/', '/team/1610612757/blazers/']]